In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#chargement fichiers
dfa_full = pd.read_csv("AllMoviesDetailsCleaned.csv", sep=";", low_memory=False, parse_dates=["release_date"])
dfb_full = pd.read_csv("AllMoviesCastingRaw.csv", sep=";", low_memory=False)

In [60]:
#Fusion fichiers par ids communes
df_full = pd.merge(dfa_full, dfb_full, on='id')
#Création fichier avec les seules données pertinentes
df_rel = df_full[['id','director_name','release_date', 'original_language','vote_count','vote_average','popularity','genres']].sort_values(by='director_name',ascending=True)
#Retraitement de la popularité : quand il y a des puissances (e-)ou des virgules on force à zéro
unreadable_popu = np.logical_or(df_rel['popularity'].str.contains("e-"), df_rel['popularity'].str.contains(","))
df_rel.loc[unreadable_popu,'popularity'] = 0
df_rel['popularity']=df_rel['popularity'].astype(float)

In [9]:
#Les 01-01-1900 sont aussi visiblement une valeur par défaut chez TMDB, on defaulte à 01-01-1800
serie = df_rel['release_date']==pd.Timestamp('1900-01-01')
df_rel.loc[serie,'release_date']=pd.Timestamp('1800-01-01')

#on defaulte à 01-01-1800 les valeurs nulles
serie = pd.isnull(df_rel['release_date'])
df_rel.loc[serie,'release_date']=pd.Timestamp('1800-01-01')

In [10]:
#extraction de l'année et réindexation
df_rel['release_date2']=df_rel['release_date'].map(lambda x:x.year)
#On réindexe
df_rel2 = df_rel.reset_index(drop = True)
df = df_rel2

In [11]:
#Split par genre individuel de chaque film
df['genres']=df['genres'].fillna(0)
all_genres = ('Drama','Crime','Action','Documentary','Adventure','Animation','Comedy',
              'Mystery','Horror','Western','Science Fiction',
              'Thriller','Romance','Fantasy','War','Family','Music','History','TV Movie','Foreign')

for genre in all_genres:
    df[genre]=df['genres'].str.contains(genre,na=False)
    df[genre]=df[genre].map({True:1, False:0})

In [13]:
#Decompte par langage principal
all_lang = ('en','de','fr','es','pt','ja','it','zh','sv','da')

for lang in all_lang:
    df[lang]=df['original_language'].str.contains(lang,na=False)
    df[lang]=df[lang].map({True:1, False:0})
    

# Petit "trick" pour compter le nb de films, une fois le df groupé par director name dans "other" il y aura le nb de films
df["other"]=1
df_grouped = df.groupby("director_name").sum()

#calcul de la popularité moyenne (on inclut les valeurs nulles)
df_grouped["popularity"]=df_grouped["popularity"]/df_grouped["other"]

# Les films d'un réal en langage "other" = (somme de ses films) - (somme de ses films dans un langage principal)
for lang in all_lang:
    #On modifie le nb de film dans des langages "autre" en soustrayant chaque nb de film ds chaque langage
    df_grouped["other"]=df_grouped["other"]-df_grouped[lang]


id                            int64
director_name                object
release_date         datetime64[ns]
original_language            object
vote_count                    int64
vote_average                float64
popularity                   object
genres                       object
release_date2                 int64
dtype: object

In [7]:
#valeurs retenues pour chaque réal: nombre de films, liste des années de sortie des films
dir_time=dict()
x = 0
#parcours de la liste de réals
while x < len(df):
    dir_name = df.loc[x,'director_name']
    dir_time[dir_name] = {'movie_nb':0,'movie_nb_inclnodate':0, 'dates':[]}
    #les réals sont par ordre alphabétique, on parcourt la liste de films de chacun
    while df.loc[x,'director_name'] == dir_name :
        dir_time[dir_name]['movie_nb_inclnodate'] +=1
        if df.loc[x,'release_date2']!=1800:
            dir_time[dir_name]['movie_nb'] +=1
            #on ajoute la date à la liste si elle n'y est pas déjà
            if df.loc[x,'release_date2'] not in dir_time[dir_name]['dates']:
                dir_time[dir_name]['dates'].append(df.loc[x,'release_date2'])
            
        x += 1
        if x == len(df):
            break

#CALCUL DE LA MOYENNE
for key in dir_time:
    #valeur par défaut de 2 ans pour le 1er film
    if dir_time[key]['movie_nb']>0:
        min_date = min(dir_time[key]['dates'])
        max_date = max(dir_time[key]['dates'])
        avg=round((max_date - min_date + 2) / dir_time[key]['movie_nb'], 1)
    
    #valeur par défaut de 2 ans si les release_dates sont toutes defaultées
    else:
        min_date = 1800
        max_date = 1800
        avg=2
        
    #piste d'audit
    dir_time[key]['min_date']=min_date
    dir_time[key]['max_date']=max_date
    dir_time[key]['career_middle_date']=int((max_date+min_date)/2)
    dir_time[key]['avg_time']=avg
    
    #On vire la clé "dates" pour avoir un dictionnaire "plat"
    dir_time[key].pop('dates', None)



In [8]:
#On transforme dir_time en dataframe
df_part1 = pd.DataFrame.from_dict(dir_time, orient='index')

#Fusion des dataframes df_part1 (avec la moyenne) et df_grouped (avec les compteurs genre/langage)
df_total = pd.concat([df_part1, df_grouped], axis=1)

#Ecriture dans un fichier
df_total.to_csv("director_avg.csv",encoding='utf-8', sep=";")